In [1]:
import time
import numpy as np
import scipy.sparse as sps

In [2]:
def load_data_into_matrix(filename, n_users, n_movies, n_ratings, discrete=False):
    """
    Create a sparse matrix from the discretized ratings data.
    
    filename: string, name of the file containing the data
    n_users: int, number of users
    n_movies: int, number of movies
    n_ratings: int, number of ratings
    discrete: bool, whether to use discrete data
    """
    data = np.load(filename)
    if discrete:
        ratings = np.ones(n_ratings)
    else:
        ratings = data[:, 2]
    users = data[:, 0] - 1
    movies = data[:, 1] - 1
    coo = sps.coo_matrix((ratings, (users, movies)), shape=(n_users, n_movies), dtype=np.int32)
    return coo


def cosine_sim(u, v):
    """
    Calculate the cosine similarity between vectors u and v.
    
    u: array, 1st vector
    v: array, 2nd vector
    """
    p1_p2 = np.dot(u, v)
    norm_p1_norm_p2 = np.sqrt(np.sum(np.square(u)) * np.sum(np.square(v)))
    if norm_p1_norm_p2 == 0:
        return 0.5
    else:
        return 1 - np.arccos(p1_p2 / norm_p1_norm_p2) / np.pi


def make_random_projections(n_projections, n_movies):
    """
    Make a sparse matrix containing random projections in N_movies-dimensional space.
    
    n_projections: int, number of projections
    n_movies: int, number of movies
    """
    flat_v = np.random.choice([-1, 1], size=(n_projections * n_movies))
    mesh = np.array(np.meshgrid(np.arange(n_projections), np.arange(n_movies))).T.reshape(-1,2)
    projection_indices, movie_indices = mesh[:, 0], mesh[:, 1]
    return sps.csr_matrix((flat_v, (projection_indices, movie_indices)), 
                          shape=(n_projections, n_movies), dtype=np.float32)

def unique_pairs_from_array(arr):
    """
    Finds all unique pairs in an array that does not contain duplicates.
    
    arr: array, array that does not contain duplicate values
    """
    pairs = []
    n = len(arr)
    for idx, val in enumerate(arr[:-1]):
        pairs_for_single_value = np.stack((np.tile(val, n - idx - 1), arr[idx + 1:]), axis=1)
        pairs.append(pairs_for_single_value)
    return np.concatenate(pairs, axis=0)

In [3]:
def create_signatures(sps_rating_matrix, n_projections):
    """
    Create signatures for each user by taking the dot product with random projections.
    
    rating_matrix: scipy.sparse.coo_matrix, sparse matrix containing the ratings
    n_projections: int, the number of projections vectors to use for the signatures
    """
    t0 = time.time()

    # Create a number of random projection vectors in N_movie-dimensional space
    n_movies = sps_rating_matrix.shape[1]
    v = make_random_projections(n_projections, n_movies)

    # Make a signature matrix by taking the dot product of the rating matrix with the random projections matrix
    # Convert all positive values to 1 and all negative values to 0 
    # [n_projections, n_movies] @ [n_movies, n_users] -> [n_projections, n_users]
    signatures = ((v * sps_rating_matrix.T).toarray() > 0).astype(np.int32)

    print('Create signatures time:', time.time() - t0)
    
    return signatures

def find_candidate_pairs(signatures, n_bands, projections_per_band):
    """
    Find candidate pairs that have the same signatures in a certain band. Employs the LSH algorithm.
    
    signatures: array, matrix containing signatures for each user with shape [n_projections, n_users]
    n_bands: int, the number of bands to use for LSH
    projections_per_band: int, the number of projections in each band
    """
    t0 = time.time()
    n_projections = n_bands * projections_per_band

    binary_array = np.array([2**i for i in np.arange(projections_per_band)[::-1]])
    candidate_pairs = []

    for b in np.arange(n_bands):
        # Select a band from the signatures
        start_of_band = b * projections_per_band
        end_of_band = min(start_of_band + projections_per_band, n_projections)
        band = signatures[start_of_band:end_of_band, :]

        # The signature of each user is an array of 1s and 0s. We can treat this as a binary number 
        # to give each user an integer value as its signature.
        band = np.dot(binary_array, band).T

        # Find all unique signatures in the band
        unique_signatures = np.unique(band)

        # For each unique signature, find the indices where that signature occurs
        for u in unique_signatures:
            indices_of_unique_signature = np.where(band == u)[0]

            # If more than 1 index is found, it means that multiple users have the same signature
            # and are likely to be similar
            if indices_of_unique_signature.shape[0] > 1:
                pairs_with_unique_signature = unique_pairs_from_array(indices_of_unique_signature)
                candidate_pairs.append(pairs_with_unique_signature)

    # Concatenate the candidate pairs found for all bands and unique signatures
    # and remove duplicate pairs
    if len(candidate_pairs) > 0:
        candidate_pairs = np.unique(np.concatenate(candidate_pairs, axis=0), axis=0).astype(np.int32)
    else:
        candidate_pairs = np.array(candidate_pairs).astype(np.int32)
        print('No pairs found!')

    print('Find candidate pairs time:', time.time() - t0)
    
    return candidate_pairs


def find_signature_pairs(candidate_pairs, signatures, threshold=0.73):
    """
    Find the pairs with a signature similarity above a certain threshold from an array of candidate pairs.
    
    candidate_pairs: array, array containing the user indices of candidate pairs with shape [n_pairs, 2]
    signatures: array, matrix containing signatures for each user with shape [n_projections, n_users] 
    threshold: float, if the fraction of identical signatures between two users is above this treshold,
    then they are selected as a signature pair. Should be a value between 0 and 1.
    """
    t0 = time.time()
    
    n_projections = signatures.shape[0]
    
    signature_similarities = []
    signatures = np.where(signatures == 0, -1, signatures)
    for user_1, user_2 in candidate_pairs:
        signature_similarities.append(np.dot(signatures[:, user_1], signatures[:, user_2]))
        
    sig_threshold = n_projections * (2 * threshold - 1)
    signature_pairs = candidate_pairs[np.array(signature_similarities) > sig_threshold]
    
    print('Find signature pairs time:', time.time() - t0)
    
    return signature_pairs

    
def find_cosine_pairs(signature_pairs, csr, threshold=0.73):
    """
    Find the pairs with a cosine similarity above a certain threshold from an array of signature pairs.
    
    csr: scipy.sparse.csr_matrix, sparse row matrix where each row contains the movie ratings of a user
    candidate_pairs: array, array containing the user indices of candidate pairs with shape [n_pairs, 2]
    threshold: float, if the cosine similarity of two users is above this threshold, then they are 
    selected as a cosine pair. Should be a value between 0 and 1.
    """
    
    t0 = time.time()
    
    cosine_similarities = []
    for user_1, user_2 in signature_pairs:
        u1_ratings = csr[user_1].toarray()[0]
        u2_ratings = csr[user_2].toarray()[0]
        cosine_similarities.append(cosine_sim(u1_ratings, u2_ratings))

    cosine_pairs = signature_pairs[np.array(cosine_similarities) > threshold]
    
    print('Find cosine pairs time:', time.time() - t0)
    
    return cosine_pairs


def find_similar_users(sps_rating_matrix, n_bands, projections_per_band, threshold):
    """
    Find similar user pairs in a sparse rating matrix. This is done in 4 steps.
    
    Step 1: Calculate a number of signatures for each user
    Step 2: Find candidate pairs by finding users that have the same signatures in a certain band
    Step 3: Filter the candidate pairs by taking the pairs where the fraction of similar signatures
    is above a certain threshold
    Step 4: Filter the remaining pairs by taking the pairs that have a cosine similarity above a 
    certain threshold.
    
    sps_rating_matrix: scipy.sparse.csr_matrix, sparse row matrix where each row contains 
    the movie ratings of a user
    n_bands: int, the number of bands to use for LSH
    projections_per_band: int, the number of projections in each band
    threshold: float, should be a value between 0 and 1.
    """
    
    print('number of bands:', n_bands)
    print('number of projections per band:', projections_per_band)
    print('threshold:', threshold)

    n_projections = n_bands * projections_per_band

    signatures = create_signatures(sps_rating_matrix, n_projections)
    
    candidate_pairs = find_candidate_pairs(signatures, n_bands, projections_per_band)
    n_candidate_pairs = candidate_pairs.shape[0]
    print('Number of candidate pairs:', n_candidate_pairs)
    
    signature_pairs = find_signature_pairs(candidate_pairs, signatures, threshold)
    n_signature_pairs = signature_pairs.shape[0]
    print('Number of signature pairs:', n_signature_pairs)
    
    cosine_pairs = find_cosine_pairs(signature_pairs, sps.csr_matrix(sps_rating_matrix), threshold)
    n_cosine_pairs = cosine_pairs.shape[0]
    print(f'Number of pairs with cos_sim > {threshold}:', n_cosine_pairs)
    
    return cosine_pairs

In [4]:
discrete = False
seed = 42
threshold = 0.73

np.random.seed(seed)

filename = 'user_movie_rating.npy'
n_users = 103703
n_movies = 17770
n_ratings = 65225506

coo = load_data_into_matrix(filename, n_users, n_movies, n_ratings, discrete)

In [5]:
n_bands = 5
projections_per_band = 20

t_start = time.time()

similar_users = find_similar_users(coo, 
                                   n_bands,
                                   projections_per_band,
                                   threshold)

run_time = time.time() - t_start
print('Total run time', int(run_time // 60), 'min', int(run_time % 60), 'sec')

if discrete:
    output_file = 'dcs.txt'
else:
    output_file = 'cs.txt'

np.savetxt(output_file, similar_users, fmt='%i', delimiter=',')

number of bands: 5
number of projections per band: 20
threshold: 0.73
Create signatures time: 22.650811672210693
Find candidate pairs time: 15.31496810913086
Number of candidate pairs: 590147
Find signature pairs time: 2.0485174655914307
Number of signature pairs: 89456
Find cosine pairs time: 16.754043579101562
Number of pairs with cos_sim > 0.73: 39
Total run time 0 min 57 sec


In [6]:
# from multiprocessing import Pool, Lock

# def find_similar_user_pairs_multip(csr, csc, n_bands, projections_per_band_arr, threshold, discrete, seed):
#     for projections_per_band in projections_per_band_arr:
#         t_start = time.time()
        
#         n_candidate_pairs, n_similar_pairs = find_similar_user_pairs(csr, 
#                                                                      csc, 
#                                                                      n_bands, 
#                                                                      projections_per_band, 
#                                                                      threshold)
#         run_time = time.time() - t_start
#         print(25*'--')
        
#         mp_lock.acquire()
#         with open('runs_log.txt', 'a') as run_file:
#             run_file.write(f'{n_bands},{projections_per_band},{n_candidate_pairs},' + 
#                            f'{n_similar_pairs},{run_time},{discrete},{seed}\n')
#         mp_lock.release()

In [7]:
# n_bands_arr = np.arange(10,11)
# projections_per_band_arr = np.arange(12, 13)

# if multip:
#     mp_lock = Lock()
#     params = [(csr, 
#            csc, 
#            n_bands, 
#            projections_per_band_arr,
#            threshold,
#            discrete,
#            seed) for n_bands in n_bands_arr]
#     pool = Pool(len(n_bands_arr))
#     fold_errors = pool.starmap(find_similar_user_pairs_multip, params)
#     pool.close()
#     pool.join()
# else:
#     for n_bands in n_bands_arr:
#         for projections_per_band in projections_per_band_arr:
#             t_start = time.time()

#             n_cand_pairs, n_sign_pairs, n_cos_pairs = find_similar_user_pairs(csr, 
#                                                                               csc, 
#                                                                               n_bands, 
#                                                                               projections_per_band,
#                                                                               threshold)

#             run_time = time.time() - t_start
#             print(25*'--')

#             with open('runs_log.txt', 'a') as run_file:
#                 run_file.write(f'{n_bands},{projections_per_band},{n_cand_pairs},' + 
#                                f'{n_sign_pairs},{n_cos_pairs},{run_time},{discrete},{seed}\n')
        